# Starbucks in San Mateo.

## 1º Import the libraries 

In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import sys
import pandas as pd
from functools import reduce
import operator
import geopandas as gpd
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry

## 2º Connect with the api.

In [2]:
load_dotenv()

True

In [3]:
token1 = os.getenv("token1")
token2 = os.getenv("token2")

In [4]:
url_query = 'https://api.foursquare.com/v2/venues/explore'

In [5]:
parametros = {"client_id" : token1,
              "client_secret" : token2,
              "v": "20180323",
              "ll": "37.566879,-122.323895",
              "query": "Starbucks",
              "limit": 5,
             "radius":1000}

## 2º Query.

In [6]:
resp = requests.get(url = url_query, params = parametros)
data = json.loads(resp.text)
#data

In [7]:
data.keys()

dict_keys(['meta', 'response'])

In [8]:
decoding_data = data.get("response")
#decoding_data

In [9]:
'''for k,v in decoding_data.items():
    print(k,v,"\n")'''

'for k,v in decoding_data.items():\n    print(k,v,"\n")'

In [10]:
decoded = decoding_data.get("groups")[0]
#decoded

In [11]:
starbucks_san_mateo = decoded.get("items")

In [12]:
'''for k,v in starbucks_san_mateo[0].items():
    print(k,"-->",v,"\n")'''

'for k,v in starbucks_san_mateo[0].items():\n    print(k,"-->",v,"\n")'

In [13]:
'''from functools import reduce
import operator'''

'from functools import reduce\nimport operator'

In [14]:
m_name = ["venue","name"]
m_latitud = ["venue","location","lat"]
m_longitud = ["venue","location","lng"]

https://docs.python.org/3/library/operator.html

In [15]:
def getFromDict(dictionary,maps):
    return reduce (operator.getitem,maps,dictionary)

In [16]:
print(getFromDict(starbucks_san_mateo[0],m_name))

Starbucks


In [17]:
lst_starbucks_san_mateo = []
for dic in starbucks_san_mateo:
    starbucks = {}
    starbucks["name"] = getFromDict(dic,m_name)
    starbucks["latitud"] = getFromDict(dic,m_latitud)
    starbucks["longitud"] = getFromDict(dic,m_longitud)
    lst_starbucks_san_mateo.append(starbucks)
    
#lst_starbucks_san_mateo

In [18]:
df_starbucks_san_mateo = pd.DataFrame(lst_starbucks_san_mateo)
df_starbucks_san_mateo.head()

,name,latitud,longitud
0,Starbucks,37.562890,-122.324005
1,Starbucks,37.571471,-122.313622
2,Starbucks,37.579827,-122.333364
3,Starbucks,37.554230,-122.306329
4,Starbucks,37.576523,-122.348110


In [19]:
'''import geopandas as gpd'''

'import geopandas as gpd'

In [20]:
gdf_starbucks_san_mateo = gpd.GeoDataFrame(df_starbucks_san_mateo, geometry = gpd.points_from_xy(df_starbucks_san_mateo.longitud,df_starbucks_san_mateo.latitud))
gdf_starbucks_san_mateo

,name,latitud,longitud,geometry
0,Starbucks,37.562890,-122.324005,POINT (-122.32400 37.56289)
1,Starbucks,37.571471,-122.313622,POINT (-122.31362 37.57147)
2,Starbucks,37.579827,-122.333364,POINT (-122.33336 37.57983)
3,Starbucks,37.554230,-122.306329,POINT (-122.30633 37.55423)
4,Starbucks,37.576523,-122.348110,POINT (-122.34811 37.57652)


In [21]:
'''from pymongo import MongoClient,GEOSPHERE
import shapely.geometry'''

'from pymongo import MongoClient,GEOSPHERE\nimport shapely.geometry'

In [22]:
gdf_starbucks_san_mateo["geometry"] = gdf_starbucks_san_mateo["geometry"].apply(lambda x: shapely.geometry.mapping(x))
gdf_starbucks_san_mateo

/usr/local/lib/python3.9/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,name,latitud,longitud,geometry
0,Starbucks,37.562890,-122.324005,"{'type': 'Point', 'coordinates': (-122.324005,..."
1,Starbucks,37.571471,-122.313622,"{'type': 'Point', 'coordinates': (-122.3136221..."
2,Starbucks,37.579827,-122.333364,"{'type': 'Point', 'coordinates': (-122.3333638..."
3,Starbucks,37.554230,-122.306329,"{'type': 'Point', 'coordinates': (-122.306329,..."
4,Starbucks,37.576523,-122.348110,"{'type': 'Point', 'coordinates': (-122.34811, ..."


In [23]:
type(gdf_starbucks_san_mateo.loc[0,"geometry"])

dict

## 3º Import the query to Mongo in a new collection.

In [24]:
client = MongoClient() 
db = client.ironhack
collection = db.starbucks 
collection.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [25]:
data = gdf_starbucks_san_mateo.to_dict(orient='records')
data[0]

{'name': 'Starbucks',
 'latitud': 37.56289,
 'longitud': -122.324005,
 'geometry': {'type': 'Point', 'coordinates': (-122.324005, 37.56289)}}

In [26]:
collection.insert_many(data)

## 4º Select the nearest Starbucks. 

In [27]:
#Call to the Collection.
db.starbucks

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'starbucks')

In [28]:
filt = {"geometry":{"$near":{"type":"Point","coordinates":[-122.323895,37.566879]}}}
proyect = {"_id":0,"name":1,"latitud":1,"longitud":1}

result = db.starbucks.find(filt,proyect).limit(1)
starbuck = list(result)

In [29]:
starbuck

[{'name': 'Starbucks', 'latitud': 37.56289, 'longitud': -122.324005}]

## 5º Map with all Starbucks.


In [30]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd

In [31]:
inicial_lat = 37.566879
inicial_long = -122.323895

In [32]:
maps = folium.Map(location = [inicial_lat,inicial_long], zoom_start=15)

In [33]:
San_Mateo = Marker(location = [inicial_lat,inicial_long],tooltip="New office")
San_Mateo.add_to(maps)

In [34]:
for i,row in df_starbucks_san_mateo.iterrows():
    starbucks = {
        "location" : [row["latitud"],row["longitud"]],
        "tooltip" : row["name"]    
    }
    if row["latitud"] == 37.56289 and row["longitud"]== -122.324005 :
        icon = Icon(color = "green",
                    prefix = "fa",
                    icon = "coffee",
                    icon_color = "black"
        )
    else:
        icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "coffee",
                    icon_color = "black"
        )
 
    Marker(**starbucks,icon = icon).add_to(maps)
maps